In [ ]:
from pymongo import MongoClient
import json
from bson import ObjectId
import os

mongo_conn_str = os.environ["FESH_MONGO"]
#Create a db connection
client = MongoClient(mongo_conn_str)

# First define the database name
dbname = client['feshdb']

def decoder(dct):
    "conver the _id type from Mongo to str"
    for k, v in dct.items():
        if k == '_id':
            dct[k] = str(v)
            break
    return dct

In [ ]:
def create_family(name=None, position=None):
    """
    Add a single family member
    """
    if not name and not position:
        return "Please supply both 'name' and 'position'"

    collection = dbname['family']
    fam = {"name":name, "position": position}

    try:
        collection.insert_one(fam)
        return "family member added."
    except Exception as e:
        return f"Error writing to db: {str(e)}"
    
def get_family_by_id(_id=None):
    if not _id:
        return "'_id' missing."

    collection = dbname['family']

    try:
        collection = dbname['family']
        results = collection.find({"_id":ObjectId(_id)})
        if not results:
            return "No match found."

        results = [decoder(line) for line in results]
        return results

    except Exception as e:
        return f"Error retrieving data: {str(e)}"
    
def get_family(_id=None):
    """
    Return all memebrs of the family collection
    """
    collection = dbname['family']

    try:
        collection = dbname['family']
        results = collection.find()
        if not results:
            return "No match found."

        results = [decoder(line) for line in results]
        return results

    except Exception as e:
        return f"Error retrieving data: {str(e)}"

In [ ]:
create_family(name="Bonnie Clydet", position="DaSteal")

In [ ]:
get_family_by_id("6539761d80907313d36f2a19")

In [ ]:
get_family()

In [ ]:
#Helper Functions
def payloader(event):
    if not event:
        body = {"message": "Empty payload."}
        response = {"statusCode": 400, "body":json.dumps(body, default=str), "headers":cors_headers}
        return False, response
    
    event_body = event.get('body')
    if not event_body:
        body = {"message": "No payload to process."}
        response = {"statusCode": 400, "body":json.dumps(body, default=str), "headers":cors_headers}
        return False, response

    payload = json.loads(event['body'])

    return True, payload


def create_family_handler(event, context):
    """
    Add a single family member
    """
    
    #Grab and process incoming payload
    ok, payload = payloader(event)
    
    if not ok:
        return payload
    
    required_params = ["name", "position"]
    received_params = [line for line in payload.keys()]
    for item in required_params:
        if item not in received_params:
            body = {"message":f"Please supply {item}"}
            response = {"statusCode": 400, "body":json.dumps(body, default=str), "headers":cors_headers}
            return response

    collection = dbname['family']

    try:
        collection.insert_one(fam)
        body = {"message": "family member added."}
        response = {"statusCode": 200, "body":json.dumps(body, default=str), "headers":cors_headers}
        return response
    except Exception as e:
        body = {"message":f"Error writing to db: {str(e)}"}
        response = {"statusCode": 400, "body":json.dumps(body, default=str), "headers":cors_headers}
        return response
    
def get_family_by_id_handler(event, context):
    #Grab and process incoming payload
    ok, payload = payloader(event)
    
    if not ok:
        return payload
    
    if "_id" not in payload.keys():
            body = {"message":"'_id' missing."}
            response = {"statusCode": 400, "body":json.dumps(body, default=str), "headers":cors_headers}
            return response

    collection = dbname['family']
    fam = {"name":name, "position": position}

    try:
        collection = dbname['family']
        results = collection.find({"_id":ObjectId(_id)})
        if not results:
            body = {"message":"No match found."}
            response = {"statusCode": 400, "body":json.dumps(body, default=str), "headers":cors_headers}
            return response
        
        results = [decoder(line) for line in results]
        body = {"message":"successful", "data":result}
        response = {"statusCode": 200, "body":json.dumps(body, default=str), "headers":cors_headers}
        return response
    except Exception as e:
        body = {"message":f"Error retrieving data: {str(e)}"}
        response = {"statusCode": 400, "body":json.dumps(body, default=str), "headers":cors_headers}
        return False, response
    
    
def get_family_handler(event, context):
    """
    Return all memebrs of the family collection
    """
    collection = dbname['family']
    fam = {"name":name, "position": position}

    try:
        collection = dbname['family']
        results = collection.find()
        if not results:
            body = {"message":"No match found."}
            response = {"statusCode": 400, "body":json.dumps(body, default=str), "headers":cors_headers}
            return response
        
        results = [decoder(line) for line in results]
        body = {"message":"successful", "data":result}
        response = {"statusCode": 200, "body":json.dumps(body, default=str), "headers":cors_headers}
        return response
    except Exception as e:
        body = {"message":f"Error retrieving data: {str(e)}"}
        response = {"statusCode": 400, "body":json.dumps(body, default=str), "headers":cors_headers}
        return False, response